In [4]:
import re
import json
import locale
import requests
import pandas as pd
from mplogger import Logger
from datetime import datetime
from bs4 import BeautifulSoup
from collections import Counter
from html2tei import parse_date

from warcio.archiveiterator import ArchiveIterator
from webarticlecurator import WarcCachingDownloader

from blikk_specific import get_meta_from_articles_spec as get_meta

<h3>TODO:</h3>

- Mi van a pulse_embedekkel - meg kell nézni, hogy mindig van e alattuk iframe, ha igen akkor nem kell beagyazott_tartalomként jelölni
- Create BLACKLIST warc to check if really missing - urls from log [DONE]
- Invalid ref [STARTED]
- Invalid p: kérdéses, hogy kell-e a tag maga mivel ajánló: https://www.blikk.hu/aktualis/belfold/friss-hirek-eb-donto-pfizer-bognar-velence/b9v102q [DONE]
- Check if sponsoredBlock link has been removed [DONE] (Question of regif.dot item links which are empty square images are still in the TEIs)
- Why is video iframe missing from this? https://www.blikk.hu/sztarvilag/sztarsztorik/fodor-zsoka-ismerkedes-udvarloi-levelek/cxdwfvl [DONE]
- Is \<section\> class="row bottomTags" okay like this?
- Check if img srcset is okay [DONE]
- Legfrissebb hírek itt -et ki kéne venni?

<h3>NOTES:</h3>

- Quiz válasz ami megjelenik miután kitöltötted: https://www.blikk.hu/sztarvilag/sztarsztorik/kviz-botrany-byealex-toth-gabi-puskas-peti-majka-szoke-zoli/nq098f4 ezeknek mi legyen a sorsa?
- Mindenképp meg kell nézni, hogy a 'Hirdetés' subtitle-t tényleg decompolta-e. Erre kell link!
- Az ilyeneknél: https://www.blikk.hu/aktualis/belfold/dunaba-csuszott-egy-auto-szentendren/fdf5gbc amikor egy mondaton belül a \<strong\> tagek egymás mellett vannak az furcsán néz ki, lehet vele valamit kezdeni? \<strong\> a \</strong\> \<strong\>kutya\</strong\>
- Json kinyerő transform_to_html [DONE]
- text & notext újratöltés pulsembed_embed kitöltéssel [DONE]

In [2]:
# Linkek
section_recommend_problem = 'https://www.blikk.hu/aktualis/kulfold/eminem-rappelve-alazta-meg-donald-trumpot/qh11yxe'
section_row_problem = 'https://www.blikk.hu/sztarvilag/agi-amerikai-versenyre-gyur/9jqxtrr'
article_recommend_problem = 'https://www.blikk.hu/aktualis/belfold/kekesteto-ho-havazas-bukk/7cb8kj9'
image_wrapper_problem = 'https://www.blikk.hu/aktualis/biszku-bela-ejjel-vitette-el-goncz-arpadot-csaladja-mellol/g56nz38'
gallery_problem = 'https://www.blikk.hu/magyarorszag-szepe-miss-world-hungary-2016/21c7mzr'
quiz_problem = 'https://www.blikk.hu/aktualis/belfold/inzultus-az-iskolaban-megalazo-helyzetek/kp0ms9r'
formitem_num_problem = 'https://www.blikk.hu/aktualis/belfold/szavazzon-melyik-halloweeni-maskara-a-legfelelmetesebb/ndgbchz'
q_problem = 'https://www.blikk.hu/aktualis/krimi/drog-nagy-britannia-csevego-alkalmazas/cbjt3ds'
promo_problem = 'https://www.blikk.hu/sztarvilag/stohl-kis-hijan-sajat-magaval-vegzett/0l455xm'
subtitle_problem = 'https://www.blikk.hu/eletmod/tippek/hogyan-kerulheto-el-a-teli-megfazas/36sk0wh'
quiz_megosztas_problem = 'https://www.blikk.hu/sztarvilag/kultura/moliere-400-eve-szuletett-irodalom/fxf50jv'  # AMIT VISSZAAD MIUTÁN KITÖLTÖTTED A QUIZT
detail_right_size_problem = 'https://www.blikk.hu/sztarvilag/sztarsztorik/kelemen-anna-teljesen-pucer-legujabb-kepen/pn8dlz9'
recept_box_problem = 'https://www.blikk.hu/receptek/konnyu-vacsora/mennyei-recept-konnyed-csirkes-lasagne/l7n9dqe'
amikor_a_quiz_kérdés_válasza_= 'https://www.blikk.hu/sztarvilag/sztarsztorik/kviz-botrany-byealex-toth-gabi-puskas-peti-majka-szoke-zoli/nq098f4'

EMPTY_1 = 'https://www.blikk.hu/aktualis/krimi/atvagtak-a-nagyszuleik-torkat-majd-hazibulit-rendeztek-tinik/jchnml5'
EMPTY_2 = 'https://www.blikk.hu/aktualis/kineveztek-fazakast-europai-szamvevonek/ymym6nb'
EMPTY_3 = 'https://www.blikk.hu/sztarvilag/gene-simmons-nem-vagyok-jeno-bacsi-interju/w5xheny'

ajanlo_a_cikk_vegen = 'https://www.blikk.hu/aktualis/belfold/friss-hirek-eb-donto-pfizer-bognar-velence/b9v102q'

aktualis_ketszer = 'https://www.blikk.hu/aktualis/indexel-is-a-bringassisak/4zk43qs'
sponsoredBlock = 'https://www.blikk.hu/eletmod/30-kilo-fogyas-csodalatos-gyogyulas-ezek-az-emberek-bebizonyitottak-hogy-semmi-sem/hlnj16t'
missing_vid = "https://www.blikk.hu/sztarvilag/sztarsztorik/fodor-zsoka-ismerkedes-udvarloi-levelek/cxdwfvl"
horvatos = "https://www.blikk.hu/sport/foci-eb-2016/veruket-adtak-a-gyozelemert-a-horvatok-modric-dontotte-el-a-torokok-elleni-merkozest/t814r9j"
mvp1 = "https://www.blikk.hu/sztarvilag/sztarsztorik/victoria-david-beckham-fia-romeo-18/wxpmz7k"
sugarhajtas = 'https://www.blikk.hu/galeria/sugarhajtasu-ruha-hadsereg-bemutatas/227h4vf'
xfaktor = 'https://www.blikk.hu/sztarvilag/sztarsztorik/x-faktor-valogato-adas-video-olah-gergo-istenes-bence/0m2dpv5'

# a végén a facebook link itt pont jó hogy van:
monjda_el_velemenyet = 'https://www.blikk.hu/aktualis/kulfold/brexit-megszuletett-a-megallapodas-az-atmeneti-idoszakrol-a-kint-dolgozo-magyarokat/mc3qm1f'

# gemadhu.hit.gemius.
gemudot = 'https://www.blikk.hu/eletmod/egeszseg/alig-paran-tudjak-a-helyes-valaszt-tudod-hogy-melyik-a-legnagyobb-szerved/7ljhv74'

In [5]:
def response_warc_record_gen(warc_filename):
    archive_base = ArchiveIterator(open(warc_filename, 'rb'))
    for rec in archive_base:
        if rec.rec_type == 'response':
            article_url = rec.rec_headers.get_header('WARC-Target-URI')
            raw_html = rec.content_stream().read()
            yield article_url, raw_html

def extract_resp_record_data(resp):
    """Extract response date, WARC ID and raw HTML from a WARC response record"""
    warc_response_date = resp.rec_headers.get_header('WARC-Date')
    if '.' in warc_response_date:
        date_format = '%Y-%m-%dT%H:%M:%S.%fZ'
    else:
        date_format = '%Y-%m-%dT%H:%M:%SZ'
    warc_response_datetime = datetime.strptime(warc_response_date, date_format)
    warc_id = resp.rec_headers.get_header('WARC-Record-ID')
    raw_html = resp.content_stream().read().decode(resp.rec_headers.get_header('WARC-X-Detected-Encoding'))

    return warc_response_datetime, warc_id, raw_html

def from_request(link):
    txt = requests.get(link).text
    return BeautifulSoup(txt, 'html.parser')

def from_warc(warc, link):
    _, _, resp = warc.get_records(link)
    warc_response_datetime, warc_id, raw_html = extract_resp_record_data(resp)
    soup = BeautifulSoup(raw_html, 'html.parser')
    return soup

In [5]:
# locale.setlocale(locale.LC_ALL, 'hu_HU.UTF-8')
def response_warc_record_gen(warc_filename):
    archive_base = ArchiveIterator(open(warc_filename, 'rb'))
    for rec in archive_base:
        if rec.rec_type == 'response':
            article_url = rec.rec_headers.get_header('WARC-Target-URI')
            raw_html = rec.content_stream().read()
            yield article_url, raw_html

#### Duplicate list

In [14]:
with open('duplicate_question_blikk.tsv', 'r') as tsv_file:
    doubles = tsv_file.read().splitlines()
    doubles = [(d.split('\t')[0],d.split('\t')[1]) for d in doubles] 

In [21]:
blikk_links = list(mvp_warc.url_index)

In [19]:
lens = []
for double in doubles:
    left = double[0]
    right = double[1]

    lens.append(len(left.split('/')))
    lens.append(len(right.split('/')))

    if len(left.split('/')) != len(right.split('/')):
        print(double)

('https://www.blikk.hu/aktualis/lezartak-a-lanchidat/6qv1vh1', 'https://www.blikk.hu/aktualis/belfold/lezartak-a-lanchidat-igy-valtoznak-a-bkv-menetrendek/rt7prtk')
('https://www.blikk.hu/o-az-uj-gorbicz/d7ejw7v', 'https://www.blikk.hu/sport/o-az-uj-gorbicz/4bgy564')
('https://www.blikk.hu/eltemettek-bacso-petert/bz0ky18', 'https://www.blikk.hu/sztarvilag/eltemettek-bacso-petert/xz72h4m')
('https://www.blikk.hu/nem-tudta-az-anya-hogy-fiat-megoltek/crnpcwk', 'https://www.blikk.hu/aktualis/nem-tudta-az-anya-hogy-fiat-megoltek/30xkqfz')
('https://www.blikk.hu/vegyen-inkabb-kolcson/l6ytezr', 'https://www.blikk.hu/eletmod/utazas/ne-sokat-koltson-vegyen-inkabb-kolcson/6z2cdm9')
('https://www.blikk.hu/aktualis/megkeseltek-egy-rendort/6v3qdwb', 'https://www.blikk.hu/aktualis/kulfold/megkeseltek-egy-rendort-forgalmas-vasutallomas-london-lezartak/bqw1jy4')
('https://www.blikk.hu/16-evesen-alig-7-kilo/k503mef', 'https://www.blikk.hu/aktualis/csoda-16-evesen-alig-7-kilo-a-kamasz/swkt28v')
('https:

In [24]:
ends = [link.split('/')[-1] for link in blikk_links]
end_lens = [len(e) for e in ends]
print(set(end_lens))

{7}


In [25]:
titles = [link.split('/')[-2] for link in blikk_links]


In [33]:
sames = [l for l in blikk_links if l.split]

576494


556794

##### same data?

In [ ]:
'https://www.blikk.hu/aktualis/sorozatbetoro-banda-lopott-a-haladas-oltozojebol/kd7gxxr'
'https://www.blikk.hu/aktualis/sorozatbetoro-banda-lopott-a-haladas-oltozojebol/ekknm8d'

#### Transform to html

In [6]:
logger = Logger()
warc_filename_mvp = '../../warcs_dir/blikk-articles_new3.warc.gz'
mvp_warc = WarcCachingDownloader(warc_filename_mvp, None, logger, just_cache=True, download_params={'stay_offline': True})

2022-04-21 11:59:02,220 INFO: Logging started
2022-04-21 11:59:02,223 INFO: Creating index for ../../warcs_dir/blikk-articles_new3.warc.gz...
2022-04-21 12:07:36,551 INFO: Index successfully created.


In [79]:
# FROM WARC
_, _, resp = mvp_warc.get_records(mvp1)
warc_response_datetime, warc_id, raw_html = extract_resp_record_data(resp)
# soup = BeautifulSoup(raw_html, 'html.parser')


In [81]:
changed_html = transform_to_html(mvp1, raw_html, logger)

done3
done2
done1
done


In [80]:
def transform_to_html(url, raw_html, warc_logger):
    _ = url, warc_logger
    print('done3')
    soup = BeautifulSoup(raw_html, 'html.parser')
    mvp_tag = soup.find('div', {'class': ['mvpLoading']})
    if mvp_tag is not None:
        print('done2')
        json_container_tag = mvp_tag.find('div', {'data-params':True, 'data-run-module':True})
        if json_container_tag is not None:
            print('done1')
            try:
                json_data = json.loads(json_container_tag['data-params'])
                if 'parameters' in json_data.keys() and 'embedCode' in json_data['parameters'].keys():
                    new_tag = BeautifulSoup(json_data['parameters']['embedCode'], 'html.parser')
                    mvp_tag.append(new_tag)
                    json_container_tag.decompose()
                    print('done')
                    return str(soup)
            except json.JSONDecodeError:
                print('TEI LOGGER PROBLEM')
    return raw_html


In [82]:
BeautifulSoup(changed_html, 'html.parser').find('div', {'class': 'mvpLoading'})

<div class="mvpLoading">
<div class="pulsembed_embed" data-author-name="victoriabeckham" data-embed-id="vGBSpGmQW" data-embed-img="" data-extra-embed-params="%7B%7D" data-height-hint="" data-provider-name="Instagram" data-run-module="pulsembed/pulsembed.init" data-sandbox="standard" data-src="//pulsembed.eu/p2em/vGBSpGmQW/"><a href="https://www.instagram.com/p/CElkS3Op-bf/?utm_source=ig_web_copy_link" target="_blank">Happy 18th birthday @romeobeckham.... We couldn’t be prouder of the young man you have become x You are beautiful inside and out and we love you so much. The sweetest, kindest soul, our everything. So many kisses from us all xxxxx @davidbeckham @brooklynbeckham @cruzbeckham #HarperSeven</a></div>
</div>

### MAIN WARC

In [3]:
logger = Logger()
warc_filename = '../../warcs_dir/blikk-articles_new3.warc.gz'
w = WarcCachingDownloader(warc_filename, None, logger, just_cache=True, download_params={'stay_offline': True})

2022-04-14 11:14:21,621 INFO: Logging started
2022-04-14 11:14:21,624 INFO: Creating index for ../../warcs_dir/blikk-articles_new3.warc.gz...
2022-04-14 11:22:56,979 INFO: Index successfully created.


In [4]:
iter_10 = response_warc_record_gen(warc_filename)

In [5]:
for i in range(10):
    u, h = next(iter_10)
    get_meta(logger, u, BeautifulSoup(h, 'lxml'))

2022-03-23 14:05:52,836 WARNING: https://www.blikk.hu/aktualis/o-a-kalyhak-asszonya/lgkqn3q: TITLE SECTION TAG NOT FOUND!
2022-03-23 14:05:52,838 WARNING: https://www.blikk.hu/aktualis/o-a-kalyhak-asszonya/lgkqn3q: DATE PUBLISHED TAG NOT FOUND!
2022-03-23 14:05:52,945 WARNING: https://www.blikk.hu/aktualis/nem-volt-otos-a-lotton-otoslotto-nyeroszamai/rps86kn: TITLE SECTION TAG NOT FOUND!
2022-03-23 14:05:52,947 WARNING: https://www.blikk.hu/aktualis/nem-volt-otos-a-lotton-otoslotto-nyeroszamai/rps86kn: DATE PUBLISHED TAG NOT FOUND!
2022-03-23 14:05:52,985 WARNING: https://www.blikk.hu/sztarvilag/zene/3-ev-utan-uj-albumon-dolgozik-adele/dxlxx1z: TITLE SECTION TAG NOT FOUND!
2022-03-23 14:05:52,986 WARNING: https://www.blikk.hu/sztarvilag/zene/3-ev-utan-uj-albumon-dolgozik-adele/dxlxx1z: DATE PUBLISHED TAG NOT FOUND!
2022-03-23 14:05:53,025 WARNING: https://www.blikk.hu/sztarvilag/sztarsztorik/ll-junior-sztarban-sztar-leszek-kiakadt/wgf7qyq: TITLE SECTION TAG NOT FOUND!
2022-03-23 14:05:

In [23]:
tag = from_request(xfaktor).find('div', {'class': ['mvpLoading']}).find('div', {'data-params':True, 'data-run-module':True})
json_data = json.loads(tag['data-params'])
new_tag = BeautifulSoup(json_data['parameters']['embedCode'], 'html.parser')
print('old:', new_tag.text)
new_tag.clear()
print('new:', new_tag.text)

old: HtmlCode

new: 


In [99]:
detal_section = soup.find('section', class_='detail')
all_immediate_detail_tags = detal_section.find_all(recursive=False)

In [128]:

detal_section = soup.find('section', class_='detail')
all_immediate_detail_tags = detal_section.find_all(recursive=False)
if len(all_immediate_detail_tags) > 1:
    check = lambda t: t.name == 'div' and 'class' in t.attrs.keys() and t['class'] == ['live_article_section']
    for f in all_immediate_detail_tags:
        if f.name == 'p' and check(all_immediate_detail_tags[-2]) is True:
            link_tag = f.find('a', {'id':True, 'rel':True, 'href':True})
            if link_tag is not None and 'blikk.hu' in link_tag['href']:
                f.decompose()




In [51]:
tc = lambda x: x.name == 'div' and 'class' in x.attrs.keys() and x['class'] == ['live_article_section']
tc(all_immediate_detail_tags[-2])

True

In [69]:
for f in reversed(all_immediate_detail_tags):
    if f.name == 'p':
        f.decompose()

In [64]:
txt = requests.get('https://www.blikk.hu/sport/foci-eb-2016/veruket-adtak-a-gyozelemert-a-horvatok-modric-dontotte-el-a-torokok-elleni-merkozest/t814r9j').text
soup = BeautifulSoup(txt, 'lxml')
soup.find('div', {'class':'mvpLoading'})

<div class="mvpLoading">
<div data-params='{"alignment":"center","publicationId":{"pulsembed":{"client":"25598d9d-16ec-4be7-9826-9224fb07155a","id":"r1njDacN"},"type":"pulsembed"},"parameters":{"embedCode":"&lt;div class=\"pulsembed_embed\" data-run-module=\"pulsembed/pulsembed.init\" data-src=\"//pulsembed.eu/p2em/r1njDacN/\" data-height-hint=\"\" data-sandbox=\"no\" data-author-name=\"\" data-embed-id=\"r1njDacN\" data-provider-name=\"postfacebook\" data-extra-embed-params=\"%7B%7D\" data-embed-img=\"\"&gt;&lt;a href=\"https://www.facebook.com/ModricLuka10/posts/1161094607276678/\" target=\"_blank\"&gt;First game approaching #Euro16 #Croatia&lt;/a&gt;&lt;/div&gt;\n","provider":"Facebook","thumb_url":"https://ocdn.eu/pulsembed/logos/facebook.png","url":"https://www.facebook.com/ModricLuka10/posts/1161094607276678/"},"platform":{"code":"pulsembed"}}' data-run-module="embeddedapp/main.app" data-type="desktop">
</div>
</div>

In [36]:
# find('a', {'id':True, 'rel':True, 'href':True})
detail = from_request(sugarhajtas).find('section', {'class': 'detail'})

[p for p in detail.find_all('p') if 'legfrissebb' in p.get_text()]


[<p>(<a href="https://www.blikk.hu/legfrissebb?utm_source=blikk&amp;utm_medium=cikk&amp;utm_campaign=legfrissebb" id="204c5c0d-f807-49b4-88ff-65bae74748df" rel="follow">A legfrissebb hírek itt</a>)</p>]

In [35]:
detail = from_request(sugarhajtas).find('section', {'class': 'detail'})
detail.find_all('p')[1].get_text()

'(A legfrissebb hírek itt)'

In [82]:
# FROM WARC
_, _, resp = mvp_warc.get_records(mvp1)
warc_response_datetime, warc_id, raw_html = extract_resp_record_data(resp)
soup = BeautifulSoup(raw_html, 'lxml')
get_meta(logger, mvp1, soup)
#[print(a.attrs) for a in all_articles]
    

defaultdict(<function html2tei.tei_utils.tei_defaultdict.<locals>.<lambda>()>,
            {'sch:url': 'https://www.blikk.hu/sztarvilag/sztarsztorik/victoria-david-beckham-fia-romeo-18/wxpmz7k',
             'sch:name': 'Az apja sármját örökölte: 18 éves lett Victoria és David Beckham „kisfia” – videó',
             'sch:datePublished': datetime.datetime(2020, 9, 1, 14, 9),
             'sch:author': ['Blikk-információ'],
             'sch:articleSection': 'Sztársztorik',
             'sch:artcleSection': None,
             'sch:keywords': ['David Beckham',
              'Victoria Beckham',
              'romeo',
              'Romeo Beckham',
              '18',
              'születésnap',
              'sztárvilág',
              'sztársztorik']})

In [83]:
soup.find('div', {'class':'mvpLoading'})

<div class="mvpLoading">
<div data-params='{"caption":"Happy 18th birthday @romeobeckham.... We couldn’t be prouder of the young man you have become x You are beautiful inside and out and we love you so much. The sweetest, kindest soul, our everything. So many kisses from us all xxxxx @davidbeckham @brooklynbeckham @cruzbeckham #HarperSeven","publicationId":{"type":"pulsembed","pulsembed":{"id":"vGBSpGmQW","client":"25598d9d-16ec-4be7-9826-9224fb07155a"}},"parameters":{"url":"https://www.instagram.com/p/CElkS3Op-bf/?utm_source=ig_web_copy_link","provider":"Instagram","thumb_url":"https://www.instagram.com/p/CElkS3Op-bf/media?size=t","embedCode":"&lt;div class=\"pulsembed_embed\" data-run-module=\"pulsembed/pulsembed.init\" data-src=\"//pulsembed.eu/p2em/vGBSpGmQW/\" data-height-hint=\"\" data-sandbox=\"standard\" data-author-name=\"victoriabeckham\" data-embed-id=\"vGBSpGmQW\" data-provider-name=\"Instagram\" data-extra-embed-params=\"%7B%7D\" data-embed-img=\"\"&gt;&lt;a href=\"https:

In [84]:
find_word = '"url":"'
index1 = soup.find('div', {'class':'mvpLoading'}).div['data-params'].find(find_word) + len(find_word)
index2 = soup.find('div', {'class':'mvpLoading'}).div['data-params'][index1:].find('"') +1 
soup.find('div', {'class':'mvpLoading'}).div['data-params'][index1:index1+index2]

'https://www.instagram.com/p/CElkS3Op-bf/?utm_source=ig_web_copy_link"'

In [85]:
mvp_string = str(soup.find('div', {'class':'mvpLoading'}))

In [86]:
soup2 = BeautifulSoup(mvp_string, 'lxml')
def excluded_tags_spec(tag):
    tag_attrs = tag.attrs
    if tag.name == 'div' and 'data-embed-id' in tag_attrs.keys():
        tag_attrs['data-embed-id'] = '@DATA-EMBED-ID'

    if tag.name == 'div' and tag_attrs == {'class': ['mvpLoading']}:
        link_container_tag = tag.find('div', {'data-run-module':True, 'data-params':True})
        if link_container_tag is not None:
            url_beginning = '"url":"'
            index1 = link_container_tag['data-params'].find(url_beginning) + len(url_beginning)
            index2 = link_container_tag['data-params'][index1:].find('"')
            link_from_json_string = link_container_tag['data-params'][index1:index1+index2]
            tag['href'] = link_from_json_string
    return tag

excluded_tags_spec(soup2.find('div', {'class':'mvpLoading'}))

<div class="mvpLoading" href="https://www.instagram.com/p/CElkS3Op-bf/?utm_source=ig_web_copy_link">
<div data-params='{"caption":"Happy 18th birthday @romeobeckham.... We couldn’t be prouder of the young man you have become x You are beautiful inside and out and we love you so much. The sweetest, kindest soul, our everything. So many kisses from us all xxxxx @davidbeckham @brooklynbeckham @cruzbeckham #HarperSeven","publicationId":{"type":"pulsembed","pulsembed":{"id":"vGBSpGmQW","client":"25598d9d-16ec-4be7-9826-9224fb07155a"}},"parameters":{"url":"https://www.instagram.com/p/CElkS3Op-bf/?utm_source=ig_web_copy_link","provider":"Instagram","thumb_url":"https://www.instagram.com/p/CElkS3Op-bf/media?size=t","embedCode":"&lt;div class=\"pulsembed_embed\" data-run-module=\"pulsembed/pulsembed.init\" data-src=\"//pulsembed.eu/p2em/vGBSpGmQW/\" data-height-hint=\"\" data-sandbox=\"standard\" data-author-name=\"victoriabeckham\" data-embed-id=\"vGBSpGmQW\" data-provider-name=\"Instagram\" d

<h1>MISC

In [5]:
with open('not_in_counts.json', 'r') as infile:
    not_in_counts = json.load(infile)

In [17]:
[l for l in all_blikk_urls if l.split('/')[3] not in not_in_counts.keys() and l.split('/')[3] not in SUBJECT_DICT.keys()]

[]

In [21]:
[a for a in not_in_counts.keys() if 'auto' in a]

['auto',
 'autotipp',
 'vonattal-utkozott-egy-auto',
 'evszazados-autocsoda',
 'ovatosan-mossuk-regi-auto',
 'ime-a-jovo-autoja',
 'kiteptek-a-falbol-az-automatat',
 'ragyujtottak-az-autot-egy-biztonsagi-orre',
 'kertbe-szallt-az-auto',
 'auto-vagodott-a-sportarenaba',
 'penzze-tett-meseauto',
 'ormanyos-automosas',
 'kutyautok',
 'szembejottek-az-autopalyan',
 'teglaval-tamad-autokra',
 'sajat-autokkal-a-borok-utjan',
 'autoszerelesben-is-bajnok',
 'porig-egett-egy-allo-auto',
 'hamilton-uj-autoja',
 'autostoppal-a-vilag-korul',
 'ezt-a-tuzoltoautot-epitettuk-otthon',
 'luxusautoval-a-bolhapiacon',
 'ime-a-vilag-legdragabb-autoja',
 'sajat-autoja-utotte-el',
 'felborult-autojaval-a-cserbenhagyo',
 'halal-az-autopalyan',
 'mit-keres-itt-auto',
 'autot-nyertem-a-blikk-kel',
 'automata-adja-a-tejet',
 '500-loeros-autok-csataja',
 'eletveszelyesek-a-tuzoltoautok',
 'etios-ez-a-tipus-del-amerika-egyik-kedvenc-autoja',
 'megharomszorozott-auto-elettartam',
 'mutatjuk-hogy-melyik-a-google-el

In [26]:
url = "https://www.blikk.hu/sztarvilag/sztarsztorik/igy-sikerult-gorbe-noranak-feldolgoznia-a-gyaszt-sri-lankan/dyed2nt"
raw_html = requests.get(url).text
soup = BeautifulSoup(raw_html, 'lxml')

In [27]:
soup.find('meta', {'property':'article:section', 'content':True})

<meta content="Sztársztorik" property="article:section"/>

In [35]:
section_path_text_split = soup.find('meta', {'name': 'kt:section-path', 'content':True})['content'].strip().split('/')
[t for t in section_path_text_split if len(t) > 1]


['sztarvilag', 'sztarsztorik']

In [43]:
# <meta property="article:published_time" content="2021-09-19 18:35:00+0100" />
# <div class="dates d-flex flex-column flex-md-row">2021. szept 19. 16:42</div>
date = root.find('div', {'class': 'dates d-flex flex-column flex-md-row'}).get_text(strip=True)
try_date = "2022. Sep 10. 11:47"
asdate = datetime.strptime(date, "%Y. %b %d. %H:%M")

In [45]:
root.find('div', {'class': 'dates d-flex flex-column flex-md-row'})

<div class="dates d-flex flex-column flex-md-row">
            2020. feb 22. 21:23
            
        </div>

In [66]:
title_tag = root.find('section', {'class': 'mainTitle'})
title_tag.find('h1').get_text(strip=True)

'Orbán Viktor ma babakocsit kormányoz: rábízták a kisunokát, bevitte a melóhelyre – fotó'

In [39]:
with open('all_blikk_section_path.tsv', 'r') as tsv:
    paths = [a.split('\t')[0] for a in tsv.read().splitlines()]

In [48]:
all_sects = []
for l in [p.split('/') for p in paths]:
    for i in l:
        all_sects.append(i)
len(set(all_sects))

74

In [14]:
SUBJECT_DICT = {'eletmod': 'életmód',
                'galeria': 'galéria',
                'allati': 'állati',
                'egeszseg': 'egészség',
                'sztarvilag': 'sztárvilág',
                'utazas': 'utazás',
                'aktualis': 'aktuális',
                'hoppa': 'hoppá',
                'karacsony': 'karácsony',
                'husvet': 'húsvét',
                'lelek': 'lélek',
                'tavaszi-megujulas': 'tavaszi-megújulás',
                'adventi-teendok': 'adventi-teendők',
                'dizajn': 'dizájn',
                'unnepi-tippek': 'ünnepi-tippek',
                'baleset-info': 'baleset-info',
                'belfold': 'belföld',
                'kulfold': 'külföld',
                'divat': 'divat',
                'velemeny': 'vélemény',
                'erotika': 'erotika',
                'kultura': 'kultúra',
                'streaming': 'streaming',
                'zene': 'zene',
                'auto': 'autó',
                'hetvegere': 'hétvégére',
                'rio-2016': 'Rio 2016',
                'sztarsztorik': 'sztársztorik',
                'blikk-tv': 'blikk-tv',
                'garazs': 'garázs',
                'geo': 'geo',
                'gasztro': 'gasztro',
                'penz': 'pénz',
                'magyar-foci': 'magyar-foci',
                'hoppa': 'hoppá',
                'panorama': 'panoráma',
                'wtf': 'wtf',
                'filmklikk': 'filmklikk',
                'mindent-bele': 'mindent-bele',
                'gyors-ebed': 'gyors-ebéd',
                'tech': 'tech',
                'forma-1': 'forma-1',
                'durva': 'durva',
                'egyeni': 'egyéni',
                'a-nyulon-tul': 'a-nyúlon-túl',
                'retro-receptek': 'retró-receptek',
                'vilag-titkai': 'világ-titkai',
                'napi-ajanlat': 'napi-ajánlat',
                'speedzone': 'speedzone',
                'teszt': 'teszt',
                'szexi': 'szexi',
                'olimpia2021': 'olimpia2021',
                'erdekes': 'érdekes',
                'veszhelyzet': 'vészhelyzet',
                'sport': 'sport',
                'foci-eb-2016': 'foci-eb-2016',
                'husveti-tippek': 'húsvéti-tippek',
                'egyeb': 'egyéb',
                'kulfoldi-foci': 'külfoldi-foci',
                'politika': 'politika',
                'tippek': 'tippek',
                'edessegek': 'édességek',
                'konnyu-vacsora': 'könnyű-vacsora',
                'menetproba': 'menetpróba',
                'tavaszi-megujulas': 'tavaszi-megújulás',
                'egeszsegkalauz': 'egészségkalaúz',
                'eb-2021': 'eb-2021',
                'katalogus': 'katalógus',
                'receptek': 'receptek',
                'valasztas-2022': 'választás-2022',
                'spanyol-foci': 'spanyol-foci',
                'krimi': 'krimi',
                
                }

# streaming, podcast, receptek, erotika 

In [134]:
next(gener)

'belfold'

sport / https://www.blikk.hu/sport/jaszapati-petra-peking-2022-teli-olimpia-elodonto-verseny/j2nj9qc
blikk-tv / https://www.blikk.hu/blikk-tv/aktualis/kolbasz-fustolt-kolbasz-jatek-verseny/7rql0yh
auto / https://www.blikk.hu/auto/autolegendak-magazin-melle-meretaranyos-automodelleket-kapunk/6hgew62
receptek / https://www.blikk.hu/receptek/hetvegere/keszitse-el-ezeket-a-kinai-recepteket-otthon/pggqex7
streaming / https://www.blikk.hu/streaming/lakers-gyozelem-rajongok/qls323q

In [29]:
# len(w.url_index) 576494
rovatok = set([u.split('/')[3] for u in list(w.url_index) if len(u.split('/')[3])])

In [30]:
megvan = list(SUBJECT_DICT.keys())
nincsmeg = [i for i in rovatok if i not in megvan]
    

In [34]:
set(nincsmeg)

{'eltuntettek-a-menekulteket-a-keletibol',
 'filmszerepre-gyur-nagy-sandor',
 'lovagga-utottek',
 'igy-hoditott-krumplis-zsak-meretu-mellevel-bogyos-maca-video',
 'mary-zsuzsi-temetese',
 'serkedi-gabi-grid-girl',
 'folytatodik-a-hokiaranycsata',
 'veszteseges-focitorna',
 'kiemelt-vedelmet-kap-az-egereszolyv',
 'folytatodik-a-furedfest',
 '2018-maxim-party-dekoltazsai',
 'a-magyar-honvedseg-megmutatta-a-fegyvereit-budaorson',
 'csillagok-heti-horoszkop-december-10-16-unnepek',
 'toronydaruval-emeltette-lakasaba-luxuskanapejat',
 'bankkartyaval-olcsobb-lehet-a-nyaralas',
 'egess-el-akar-500-kaloriat-az-intervall-treninggel',
 'uj-tarozo-a-tiszan',
 'teljes-keszultseg-a-strandon',
 'meddig-marad-a-magas-kamat',
 'oosthuizen-utese-mindent-vitt',
 'eltunt-egy-mentokutya',
 'bella-hadid-feher-felsot-viselt-melltarto-nelkul',
 'katonai-buvarok-gyakorlata',
 'szokokutnyi-orias-vizsugar-tort-elo-a-foldbol-salgotarjanban',
 'kulonbsegek',
 'a-het-legszexibb-insta-posztjai',
 'mellbedobassal-te

In [36]:
# u1 = 'https://www.blikk.hu/eletmod/tippek/hoki-parajegkorong-teli-olimpiai-jatekok/pxnyc94'
blikk_iter = response_warc_record_gen(warc_filename)
not_in_2 = []
for i in range(576494):
    if i % 1000 == 0:
        print(f'{round((i / 576494) * 100, 2)}%')
    u, _ = next(blikk_iter)
    if u.split('/')[3] not in megvan:
        not_in_2.append(u)

0.0%
0.17%
0.35%
0.52%
0.69%
0.87%
1.04%
1.21%
1.39%
1.56%
1.73%
1.91%
2.08%
2.26%
2.43%
2.6%
2.78%
2.95%
3.12%
3.3%
3.47%
3.64%
3.82%
3.99%
4.16%
4.34%
4.51%
4.68%
4.86%
5.03%
5.2%
5.38%
5.55%
5.72%
5.9%
6.07%
6.24%
6.42%
6.59%
6.77%
6.94%
7.11%
7.29%
7.46%
7.63%
7.81%
7.98%
8.15%
8.33%
8.5%
8.67%
8.85%
9.02%
9.19%
9.37%
9.54%
9.71%
9.89%
10.06%
10.23%
10.41%
10.58%
10.75%
10.93%
11.1%
11.28%
11.45%
11.62%
11.8%
11.97%
12.14%
12.32%
12.49%
12.66%
12.84%
13.01%
13.18%
13.36%
13.53%
13.7%
13.88%
14.05%
14.22%
14.4%
14.57%
14.74%
14.92%
15.09%
15.26%
15.44%
15.61%
15.79%
15.96%
16.13%
16.31%
16.48%
16.65%
16.83%
17.0%
17.17%
17.35%
17.52%
17.69%
17.87%
18.04%
18.21%
18.39%
18.56%
18.73%
18.91%
19.08%
19.25%
19.43%
19.6%
19.77%
19.95%
20.12%
20.3%
20.47%
20.64%
20.82%
20.99%
21.16%
21.34%
21.51%
21.68%
21.86%
22.03%
22.2%
22.38%
22.55%
22.72%
22.9%
23.07%
23.24%
23.42%
23.59%
23.76%
23.94%
24.11%
24.28%
24.46%
24.63%
24.81%
24.98%
25.15%
25.33%
25.5%
25.67%
25.85%
26.02%
26.19%
26.37%
26.

In [48]:
['sport', 'valasztas-2022', 'receptek', 'blikk-tv', 'auto']
not_in_rovat = [u.split('/')[3] for u in not_in_2]

In [45]:
not_in_without_dash = [i for i in not_in_rovat if '-' not in i]

In [52]:
not_in_count = sorted(Counter(not_in_rovat).items(), key=lambda x:x[1], reverse=True)

In [15]:
# METACHECKING
with open('blikk10000metas.json', 'r') as infile:
    blikkdict = json.load(infile)
    

In [22]:
authors = [d['sch:author'] for d in blikkdict.values() if 'sch:author' in d.keys()]
set([c for c in [b[0] for b in [a for a in authors if len(a) == 1]] if ' ' not in c])

In [38]:
dates = [(d['sch:url'],d['sch:datePublished']) for d in blikkdict.values()]

In [3]:
-def transform_to_html(url, raw_html, warc_logger):
-    _ = url  # , warc_logger
-
-    soup = BeautifulSoup(raw_html, 'html.parser')
-    mvp_tag = soup.find('div', {'class': ['mvpLoading']})
-    if mvp_tag is not None:
-
-        json_container_tag = mvp_tag.find('div', {'data-params':True, 'data-run-module':True})
-        if json_container_tag is not None:
-
-            try:
-                json_data = json.loads(json_container_tag['data-params'])
-                if 'parameters' in json_data.keys() and 'embedCode' in json_data['parameters'].keys():
-                    new_tag = BeautifulSoup(json_data['parameters']['embedCode'], 'html.parser')
-                    mvp_tag.append(new_tag)
-                    json_container_tag.decompose()
-                    print(mvp_tag)
-                    return str(soup)
-            except json.JSONDecodeError:
-                warc_logger.log('WARNING', f'{url}: FAILED TO TRANSFORM JSON!')
-
-    return raw_html